# IMPORTING LIBRARIES

In [26]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm as cmaps
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from dateutil.relativedelta import relativedelta
from collections import Counter, OrderedDict
import itertools
import re

print("Packages loaded!")

Packages loaded!


# Pipeline Controller

df = load_original_data()
df_cleaned = delete_unncessary_columns(df)
df_cleaned = make_columns_lowercase(df_cleaned)
df_cleaned = delete_non_legit_XDC_IDs(df_cleaned)
df_cleaned, date_columns = date_columns_transform_into_datetime(df_cleaned)
df_cleaned = delete_rows_with_nan_in_selected_cols(df_cleaned, ["schmelzen_datum", "ns1_datum"])
df_cleaned = substitute_familie_int_str(df_cleaned)
df_cleaned = add_date_month_string_columns(df_cleaned, date_columns)
df_cleaned = string_comma_to_dot_float(df_cleaned)
#df_cleaned = clean_personal_names(df_cleaned)

# READ DATA INTO DATAFRAME FROM EXISTING .CSV

In [27]:
def load_original_data():
    """
    Read production data and parse into pandas dataframe.
    """
    latest_file = r"C:\Users\p.kollhof\Documents\IRONHACK_GitHub\DA_Midterm_Project\Data\Production_Data.csv"
    df_prod_whole = pd.read_csv(latest_file, sep=";", encoding = "ISO-8859-1")
    
    return df_prod_whole

df_prod_whole = load_original_data()

# DF CLEANING

## Delete unncessary/empty colums

In [28]:
def delete_unncessary_columns(df):
    """
    Delete columns only containing NaN values or columns with effectively useless data
    """
    #try:
    cols_to_drop = []
    if any(df_prod_whole.columns.str.contains("WT1")):
        cols_to_drop_1 = [col for col in df_prod_whole.columns if "WT1" in col]
        cols_to_drop.extend(cols_to_drop_1)
    if any(df_prod_whole.columns.str.contains("WT2")):
        cols_to_drop_2 = [col for col in df_prod_whole.columns if "WT2" in col]
        cols_to_drop.extend(cols_to_drop_2)
    if any(df_prod_whole.columns.str.contains("ET2")):
        cols_to_drop_3 = [col for col in df_prod_whole.columns if "ET2" in col]
        cols_to_drop.extend(cols_to_drop_3)
    if any(df_prod_whole.columns.str.contains("FT2")):
        cols_to_drop_4 = [col for col in df_prod_whole.columns if "FT2" in col]
        cols_to_drop.extend(cols_to_drop_4)
    if any(df_prod_whole.columns.str.contains("Schneiden_Ort")):
        cols_to_drop_5 = [col for col in df_prod_whole.columns if "Schneiden_Ort" in col]
        cols_to_drop.extend(cols_to_drop_5)
    if any(df_prod_whole.columns.str.contains("Schneiden_Geraet")):
        cols_to_drop_6 = [col for col in df_prod_whole.columns if "Schneiden_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_6)
    if any(df_prod_whole.columns.str.contains("Schleifen_Ort")):
        cols_to_drop_7 = [col for col in df_prod_whole.columns if "Schleifen_Ort" in col]
        cols_to_drop.extend(cols_to_drop_7)
    if any(df_prod_whole.columns.str.contains("Schleifen_Geraet")):
        cols_to_drop_8 = [col for col in df_prod_whole.columns if "Schleifen_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_8)
    if any(df_prod_whole.columns.str.contains("NS2_Ort")):
        cols_to_drop_9 = [col for col in df_prod_whole.columns if "NS2_Ort" in col]
        cols_to_drop.extend(cols_to_drop_9)
    if any(df_prod_whole.columns.str.contains("OeO_Ort")):
        cols_to_drop_10 = [col for col in df_prod_whole.columns if "OeO_Ort" in col]
        cols_to_drop.extend(cols_to_drop_10)
    if any(df_prod_whole.columns.str.contains("OeO_Geraet")):
        cols_to_drop_11 = [col for col in df_prod_whole.columns if "OeO_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_11)
        
    else:
        raise ValueError("Columns have already been deleted!")

    cols_to_drop.extend(["Versand_Intern", "SSMA_TimeStamp", "Assembley_Teflonschlauch-Charge"])
    df.drop(cols_to_drop, axis=1, inplace=True)

    #except:
        #raise ValueError("Deleting columns was unsuccessful!")
    
    return df

df_cleaned = delete_unncessary_columns(df_prod_whole)

## Make columns lowercase and fix syntax

In [29]:
def make_columns_lowercase(df):
    
    lower_cols = [i.lower().replace(" ", "_").replace("-", "_") for i in df.columns]
    df_cleaned.columns = lower_cols    
    
    return df_cleaned

df_cleaned = make_columns_lowercase(df_cleaned)

## Remove rows with XDC-ID < `68.000`

In [30]:
def delete_rows_with_XDC_ID_pre_68k(df):
    """
    Delete all entries before XDC-ID 68.000 due to them having wrong defect IDs.
    Only applies if dataframe actually has older entries with XDC-IDs < 68.000
    """
    
    df = df[df["pdc_nummer"]>=68000]
    
    return df

#df_cleaned = delete_rows_with_XDC_ID_pre_68k(df_cleaned)

## Remove rows with non-legit XDC-ID

In [31]:
def delete_non_legit_XDC_IDs(df):
    """
    Only keep entries with legit XDC-ID.
    Anything before 44145 is not usable.
    """
    
    df = df[df["pdc_nummer"]>=44145]
    
    return df

df_cleaned = delete_non_legit_XDC_IDs(df_cleaned)

## Cast `string-dates` into `datetime-dates`

In [32]:
def date_columns_transform_into_datetime(df):
    """
    Transform all dates of string type 'YYYY-MM-DD' into dt.datetime format.
    """
    date_cols = [col for col in df.columns if "datum" in col]

    for col in date_cols:
        if df[col].dtype == "object":
            df[col] = [dt.datetime.strptime(d,'%Y-%m-%d').date() if type(d)==str 
                       else np.nan if type(d)==float
                       else d
                       for d in df[col]]
            
    return df, date_cols

df_cleaned, date_columns = date_columns_transform_into_datetime(df_cleaned)

## Remove rows without entries in selected columns

In [33]:
def delete_rows_with_nan_in_selected_cols(df, cols):
    """
    Delete rows that do not have an entry in the given columns
    """
    for col in cols:
        df[col].dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    return df
    
df_cleaned = delete_rows_with_nan_in_selected_cols(df_cleaned, ["schmelzen_datum", "ns1_datum"])

## Substitute Product-`Familie` int-values with strings *("PDC", "PDC-C", "XDC")*

In [34]:
def substitute_familie_int_str(df):
    """
    Parse int values (1,2,3) into string values ("pdc", "pdc-c", "xdc")
    """
    
    if df["familie"].dtype == "int64":
        df["familie"] = ["pdc" if fam==1
                         else "pdc-c" if fam==2
                         else "xdc" 
                         for fam in df_cleaned["familie"]]
        
    return df

df_cleaned = substitute_familie_int_str(df_cleaned)

## Create `_Months` columns

In [35]:
def add_date_month_string_columns(df, date_columns):
    """
    Convert YYYY-MM-DD dt.datetime into new 'YYYY-MM' string month column. 
    """
    
    unwanted_dates = ['ts_in_datum', 'ts_out_datum', 'rt_bqc_in_datum', 'rt_bqc_out_datum']
    date_columns_wanted = [col for col in date_columns if col not in unwanted_dates]
    date_columns_months = [col.split("_")[0] + "_month" for col in date_columns_wanted]
    #date_columns_months = [col  for col in date_columns_months]
    
    #if "schmelzen_Mmnths" in df.columns:
    for idx, datum in enumerate(date_columns_wanted):
        df[date_columns_months[idx]] = [str(d)[:-3] for d in df[datum]]
    #else:
        #raise ValueError("Months columns have already been added!")
    return df

df_cleaned = add_date_month_string_columns(df_cleaned, date_columns)

## Turn string-comma-values into dot-float-values

In [36]:
def string_comma_to_dot_float(df):
    """
    Cast string-comma-values as dot-float-values
    """
    cols_to_use = ['ns1_winkel_vorne','ns1_winkel_hinten','ns1_durchmesser_vorne','ns1_durchmesser_hinten',
               "ns2_center_variation", "ns2_front_angle","ns2_curve","ns2_15xlength",
               "ns2_parabolic_factor","oeo_oeo", "piezo_phase"] 

    for column in cols_to_use:
        df[column] = [float(str(x).replace(',','.')) for x in df[column]] 
     
    return df

df_cleaned = string_comma_to_dot_float(df_cleaned)

## Clean personal (operator) initials

In [13]:
def clean_personal_names(df):
    """
    Func descr
    """
    
    replacements = {'GD': 'GDH',
                    'MFW': 'MFE',
                    "ME": "MFE",
                    "MF": "MFE",
                    "LAE": "AL",
                    "SF": "SFZ",
                    "DC": "AC",
                    "THANH": "TT",
                    "?": np.nan}
    
    personal_cols = [col for col in df_cleaned.columns if "personal" in col]
    
    for col in personal_cols:
        op_list = []

        for op in df_cleaned["ft1_personal"]:
            if type(op)==str and op.isalpha():
                op_list.append(op.upper())
            elif op=="?":
                op_list.append(np.nan)
            elif type(op)==str and "." in op:
                op_list.append(np.nan)
            elif type(op)==str and not op.isalpha() and op!="?":
                op_list.append(op[:re.search(r'\W+', op).start()].upper())
            else:
                op_list.append(np.nan)

        op_list = [replacements.get(word, word) for word in op_list]

        df[col] = op_list
        
    return df
    
#df_cleaned = clean_personal_names(df_cleaned)

# Processing `numerical` and `categorical` data

In [83]:
def get_numerical(df):
    
    df_edanum = df_cleaned.select_dtypes(include=np.number)

    ns1_cols = [col for col in df_edanum.columns if "ns1" in col if 'geraet' not in col]
    ns2_cols = [col for col in df_edanum.columns if "ns2" in col if 'geraet' not in col]
    piezo_cols = [col for col in df_edanum.columns if "piezo" in col]
    oeo_cols=[col for col in df_edanum.columns if "oeo" in col]
    vol_cols = [col for col in df_edanum.columns if "volume" in col]
    span_cols =[col for col in df_edanum.columns if "spannung" in col]
    pulse_col = [col for col in df_edanum.columns if "puls" in col]

    columns_to_use = ns1_cols + ns2_cols + piezo_cols + oeo_cols + span_cols + pulse_col + vol_cols

    df_numerical = df_edanum[columns_to_use]
    
    return df_numerical

def get_categorical(df):
    df_categorical = df_cleaned.select_dtypes(include=['object'])
    
    cols_categorical = ["familie", 'schmelzen_personal', "schmelzen_ort", 'ns1_personal', 'schneiden_personal', 
                        'schleifen_personal',
                        'ns2_personal', 'oeo_personal', 'kleben1_personal', 'kleben2_personal',
                        'bqc_personal', 'et1_personal', 'ft1_personal', 'ek_personal']
    
    df_categorical = df_categorical[cols_categorical]
    
    df_categorical["produktion_groesse"] = df_cleaned["produktion_groesse"]
    df_categorical["art"] = df_cleaned["art"]

    return df_categorical

df_numerical = get_numerical(df_cleaned)
df_categorical = get_categorical(df_cleaned)

In [85]:
df_numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112972 entries, 0 to 112971
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ns1_winkel_vorne        112921 non-null  float64
 1   ns1_winkel_hinten       112898 non-null  float64
 2   ns1_durchmesser_vorne   112971 non-null  float64
 3   ns1_durchmesser_hinten  112971 non-null  float64
 4   ns2_center_variation    112970 non-null  float64
 5   ns2_front_angle         112969 non-null  float64
 6   ns2_curve               112969 non-null  float64
 7   ns2_15xlength           112967 non-null  float64
 8   ns2_parabolic_factor    112967 non-null  float64
 9   piezo_phase             31978 non-null   float64
 10  oeo_oeo                 112948 non-null  float64
 11  bqc_spannung            35748 non-null   float64
 12  et1_spannung            22222 non-null   float64
 13  ft1_spannung            15106 non-null   float64
 14  ek_spannung         

In [72]:
cols_categorical = ["familie", 
                    'schmelzen_personal', "schmelzen_ort", 
                    'ns1_personal', 'schneiden_personal', 'schleifen_personal',
                    'ns2_personal', 'oeo_personal', 
                    'kleben1_personal', 'kleben2_personal',
                    'bqc_personal', 'et1_personal', 'ft1_personal', 'ek_personal',
                    "kleben1_kleber_teilentnahme", "kleben2_kleber_teilentnahme"


                   ]

In [73]:
[col for col in df_cleaned.columns if "kleben" in col]

['kleben1_personal',
 'kleben1_datum',
 'kleben1_ort',
 'kleben1_geraet',
 'kleben1_kleber_charge',
 'kleben1_kleber_teilentnahme',
 'kleben1_piezzo_charge',
 'kleben2_personal',
 'kleben2_datum',
 'kleben2_ort',
 'kleben2_geraet',
 'kleben2_kleber_charge',
 'kleben2_kleber_teilentnahme',
 'kleben2_huelse1_charge',
 'kleben2_isolator_charge',
 'kleben2_huelse2_charge',
 'kleben1_shore',
 'kleben1_month',
 'kleben2_month']

In [75]:
df_cleaned[df_cleaned["ft1_volumen"]==-1]

,pdc_nummer,produktion_groesse,familie,art,kommentar_produktion,grenzwert_set,defekt_arbeitsschritt_id,defekt_grund_id,schmelzen_personal,schmelzen_datum,schmelzen_ort,schmelzen_geraet,schmelzen_kapillaren_charge,ns1_personal,ns1_datum,ns1_ort,ns1_geraet,ns1_winkel_vorne,ns1_winkel_hinten,ns1_durchmesser_vorne,ns1_durchmesser_hinten,schneiden_personal,schneiden_datum,schleifen_personal,schleifen_datum,ns2_personal,ns2_datum,ns2_geraet,ns2_center_variation,ns2_front_angle,ns2_curve,ns2_15xlength,ns2_parabolic_factor,oeo_personal,oeo_datum,oeo_oeo,kleben1_personal,kleben1_datum,kleben1_ort,kleben1_geraet,kleben1_kleber_charge,kleben1_kleber_teilentnahme,kleben1_piezzo_charge,kleben2_personal,kleben2_datum,kleben2_ort,kleben2_geraet,kleben2_kleber_charge,kleben2_kleber_teilentnahme,kleben2_huelse1_charge,kleben2_isolator_charge,kleben2_huelse2_charge,assembley_personal,assembley_datum,assembley_ort,assembley_huelse3_charge,uebergabe_datum_qc,dose,bqc_personal,bqc_datum,bqc_ort,bqc_geraet,bqc_spannung,bqc_puls,bqc_volumen,bqc_auslenkung,bqc_z_offset,bqc_kommentar,ts_in_datum,ts_out_datum,rt_bqc_in_datum,rt_bqc_out_datum,uebergabe_ss,et1_datum,et1_personal,et1_spannung,et1_puls,et1_volumen,et1_auslenkung,et1_z_offset,et1_ort,et1_geraet,et1_kopfid,et1_kanal,et1_version,et1_kommentar,coatingtyp,coatingtyp_personal,ft1_typ,ft1_datum,ft1_personal,ft1_spannung,ft1_puls,ft1_volumen,ft1_auslenkung,ft1_z_offset,ft1_ort,ft1_geraet,ft1_kopfid,ft1_kanal,ft1_version,ft1_kommentar,ek_datum,ek_personal,ek_spannung,ek_puls,ek_volumen,ek_groesse,ek_auslenkung,ek_z_offset,ek_kommentar,ek_ort,ek_geraet,ek_kopfid,ek_kanal,ek_version,versand_uebergabe_datum,versand_uebergabe_personal,versand_kunde,versand_order_nr,versand_datum,versand_personal,id_aktuelles_coating,piezo_position,piezo_phase,piezo_id,ns2_linear_factor,kleben1_shore,bqc_kopfid,bqc_kanal,bqc_version,schmelzen_month,ns1_month,schneiden_month,schleifen_month,ns2_month,oeo_month,kleben1_month,kleben2_month,assembley_month,uebergabe_month,bqc_month,et1_month,ft1_month,ek_month,versand_month
33950,123391,5.00,pdc,7,NaN,1,19.00,13.00,EF,2021-05-28,Dortmund,1.00,5203,CK,2021-05-31,Dortmund,0.00,3.00,3.00,69.30,59.50,SM,2021-06-08,SM,2021-06-08,HO,2021-06-09,0.00,0.92,0.11,2.90,6.10,29.90,HO,2021-06-10,51.30,GM,2021-06-21,Dortmund,Fritz,227.00,3.00,136.00,GM,2021-06-23,Dortmund,Fritz,163.00,3.00,0.00,1.00,0.00,GM,2021-06-24,Dortmund,0.00,2021-06-24,0.00,AL,2021-06-25,---,782300146,107.00,48.00,358.00,4.00,8.00,Dekonex hei?. Aceton,2021-06-25,2021-07-02,2021-07-02,2021-07-05,2021-07-06,2021-07-20,MB,110.00,50.00,395.00,0.00,13.00,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,2021-12-27,LE,86.00,48.00,-1.00,0.00,44.00,Berlin,139782300166,1007.00,5.00,3.18.10.4,Broken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,113.90,J24,0,"70,5",41585141.00,3.00,3.18.10.4,2021-05,2021-05,2021-06,2021-06,2021-06,2021-06,2021-06,2021-06,2021-06,2021-06,2021-06,2021-07,2021-12,,


In [68]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112972 entries, 0 to 112971
Columns: 146 entries, pdc_nummer to versand_month
dtypes: float64(58), int64(4), object(84)
memory usage: 125.8+ MB


In [67]:
df_numerical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112972 entries, 0 to 112971
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   pdc_nummer                   112972 non-null  int64  
 1   produktion_groesse           108981 non-null  float64
 2   art                          112972 non-null  int64  
 3   grenzwert_set                112972 non-null  int64  
 4   defekt_arbeitsschritt_id     112945 non-null  float64
 5   defekt_grund_id              112951 non-null  float64
 6   schmelzen_geraet             103828 non-null  float64
 7   schmelzen_kapillaren_charge  112972 non-null  int64  
 8   ns1_geraet                   103827 non-null  float64
 9   ns1_winkel_vorne             112921 non-null  float64
 10  ns1_winkel_hinten            112898 non-null  float64
 11  ns1_durchmesser_vorne        112971 non-null  float64
 12  ns1_durchmesser_hinten       112971 non-null  float64
 13 

In [56]:
df_categorical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112972 entries, 0 to 112971
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   familie             112972 non-null  object 
 1   schmelzen_personal  112972 non-null  object 
 2   schmelzen_ort       103828 non-null  object 
 3   ns1_personal        112972 non-null  object 
 4   schneiden_personal  112851 non-null  object 
 5   schleifen_personal  112871 non-null  object 
 6   ns2_personal        112771 non-null  object 
 7   oeo_personal        112711 non-null  object 
 8   kleben1_personal    37656 non-null   object 
 9   kleben2_personal    37656 non-null   object 
 10  bqc_personal        34215 non-null   object 
 11  et1_personal        26060 non-null   object 
 12  ft1_personal        18085 non-null   object 
 13  ek_personal         13820 non-null   object 
 14  produktion_groesse  108981 non-null  float64
 15  art                 112972 non-nul